# Part 2: 联邦学习简介

在上一节中，我们了解了张量指针，它创建了隐私保护深度学习所需的基础架构。在本节中，我们将看到如何使用这些基本工具来实现我们的第一个隐私保护深度学习算法：联邦学习。

作者:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

中文版译者：
- Hou Wei - github：[@dljgs1](https://github.com/dljgs1)

### 什么是联邦学习？

它是训练深度学习模型的一种简单而强大的方法。考虑一下训练数据，一般它总是某种收集过程的结果：人们（通过设备）通过记录现实世界中的事件来生成数据。通常，此数据被聚合到单个中央位置，以便您可以训练机器学习模型。而联邦学习扭转了这一局面！

你无需将训练数据带到模型（一个中央服务器），而是将模型带到训练数据（无论其位于何处）。

这个想法允许创建数据的任何人拥有数据唯一的永久副本，从而保持对有权访问该数据的人的控制。很酷吧？

# 2.1 一个玩具联合学习的例子

让我们从一个集中式训练的玩具模型开始。就像得到模型一样简单。我们首先需要：

- 玩具数据集
- 一个模型
- 用于训练模型以适合数据的一些基本训练逻辑。

注意: 如果这些API对你很陌生，请转到教程[fast.ai](http://fast.ai)学习然后再继续。

In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:

# 一个玩具数据集
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# 一个玩具模型
model = nn.Linear(2,1)

def train():
    # 训练逻辑
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) 消除之前的梯度（如果存在）
        opt.zero_grad()

        # 2) 预测
        pred = model(data)

        # 3) 计算损失
        loss = ((pred - target)**2).sum()

        # 4) 指出那些导致损失的参数（损失回传）
        loss.backward()

        # 5) 更新参数
        opt.step()

        # 6) 打印进程
        print(loss.data)

In [ ]:
train()

现在您拥有了它！我们已经以常规方式训练了基本模型。我们所有的数据都汇总到我们的本地计算机中，我们可以使用它来更新我们的模型。但是，联邦学习无法以这种方式工作。 因此，让我们修改此示例以实现联合学习方式！

所以我们需要:

- 创建一对工作机
- 获取每个工作机关于训练数据的指针
- 联邦学习的训练逻辑：
    每一步新的训练步骤:
    - 将模型发送给对应的工作机
    - 在该机的数据上完成训练
    - 拿回模型然后在下一个工作机重复

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)


In [ ]:
# 创建一对工作机

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [ ]:
# 一个玩具数据集
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# 通过以下方式获取每个工作机的训练数据的指针
# 向bob和alice发送一些训练数据
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# 初始化玩具模型
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# 将指针组织到列表中
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [ ]:
def train():
    # 训练逻辑
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW）遍历每个工作人员的数据集
        for data,target in datasets:
            
            # NEW) 将模型发送给对应的工作机
            model.send(data.location)

            # 1) 消除之前的梯度（如果存在）
            opt.zero_grad()

            # 2) 预测
            pred = model(data)

            # 3) 计算损失
            loss = ((pred - target)**2).sum()

            # 4) 指出那些导致损失的参数（损失回传）
            loss.backward()

            # 5) 更新参数
            opt.step()
            
            # NEW) 获取模型（带梯度）
            model.get()

            # 6) 打印进程
            print(loss.data)
    
# 联合平均

In [ ]:
train()

## 做得好！

瞧！我们现在正在使用联邦学习训练非常简单的深度学习模型！我们将模型发送给每个工作人员，生成新的梯度，然后将梯度带回我们的本地服务器，在此更新全局模型。在此过程中，我们永远不会看到或请求访问基础训练数据！我们保留Bob和Alice的隐私！

## 这个例子的缺陷

因此，尽管此示例是联合学习的不错介绍，但仍存在一些主要缺点。最值得注意的是，当我们调用`model.get()`并从Bob或Alice接收更新的模型时，我们实际上可以通过查看Bob和Alice的梯度来学习很多关于Bob和Alice的训练数据。在某些情况下，我们可以完美地恢复他们的训练数据！（译者注：此处属于隐私泄露攻击）

那么，该怎么办？好吧，人们采用的第一个策略是**在将多个梯度上载到中央服务器之前对多个个体进行平均**。但是，此策略将需要对张量指针对象进行更复杂的使用。因此，在下一节中，我们将花费一些时间来学习更多高级指针功能，然后我们将升级此联邦学习示例。

# 恭喜!!! 是时候加入社区了!

祝贺您完成本笔记本教程！ 如果您喜欢此方法，并希望加入保护隐私、去中心化AI和AI供应链（数据）所有权的运动，则可以通过以下方式做到这一点！

### 给 PySyft 加星

帮助我们的社区的最简单方法是仅通过给GitHub存储库加注星标！ 这有助于提高人们对我们正在构建的出色工具的认识。

- [Star PySyft](https://github.com/OpenMined/PySyft)

### 加入我们的 Slack!

保持最新进展的最佳方法是加入我们的社区！ 您可以通过填写以下表格来做到这一点[http://slack.openmined.org](http://slack.openmined.org)

### 加入代码项目!

对我们的社区做出贡献的最好方法是成为代码贡献者！ 您随时可以转到PySyft GitHub的Issue页面并过滤“projects”。这将向您显示所有概述，选择您可以加入的项目！如果您不想加入项目，但是想做一些编码，则还可以通过搜索标记为“good first issue”的GitHub问题来寻找更多的“一次性”微型项目。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 捐赠

如果您没有时间为我们的代码库做贡献，但仍想提供支持，那么您也可以成为Open Collective的支持者。所有捐款都将用于我们的网络托管和其他社区支出，例如黑客马拉松和聚会！

[OpenMined's Open Collective Page](https://opencollective.com/openmined)